In [1]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari

In [2]:
os.chdir('/home/ali/packages/s2p-lbm/')

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu
from suite3d import file_utils as flu

In [3]:
from mkl_fft import fftn, ifftn
import cupy as cp
import cupyx.scipy.fft as cufft
import scipy.fft
import cupyx.scipy.ndimage as cuimage

In [4]:
# Create the job
job = Job('/mnt/md0/runs','AH007_2022-03-28_2-4-5', create=False, overwrite=False, verbosity = 10)
tifs, si_params, exp_str = flu.find_exp(job.params['subjects_dir'], job.params['subject'],
                                        job.params['date'], job.params['expnum'])

exp_dir = flu.find_expt_file((job.params['subject'], job.params['date'], job.params['expnum'][0]),
                         'root', dirs=[job.params['subjects_dir']])
offset_files = job.get_registered_files('registered_data', 'offset')

   Found and loaded params from /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5/params.npy
SI ROI pix count in x is 182, which is impossible, setting it to 145


In [5]:
cz = (0,25)
ct = (0, 50)
cy = (None,None)
cx = (None,None)

In [6]:
offsets = [n.load(f,allow_pickle=True).item() for f in offset_files]
offset = offsets[0]
for k,v in offset.items():
    offset[k] = n.array(n.concatenate([o[k] for o in offsets],axis=1) )
    offset[k] = offset[k][cz[0]:cz[1], ct[0]:ct[1]]
dx = (offset['xms'] - n.floor(offset['xms'].mean(axis=1,keepdims=True)))[cz[0]:cz[1]]
dy = (offset['yms'] - n.floor(offset['yms'].mean(axis=1,keepdims=True)))[cz[0]:cz[1]]

In [7]:
summary = job.load_summary()
all_ops = summary['all_ops']
crosstalk_coeff = summary['crosstalk_coeff']
refs_and_masks = summary['refs_and_masks']
ref_img = summary['ref_img_3d'][cz[0]:cz[1]]
min_pix_vals = summary['min_pix_vals']
mask_mul, mask_offset, ref_2ds = n.stack([r[:3] for r in refs_and_masks],axis=1)[:,cz[0]:cz[1], cy[0]:cy[1], cx[0]:cx[1]]
mask_mul_nr, mask_offset_nr, ref_nr = n.stack([r[3:] for r in refs_and_masks],axis=1)[:,cz[0]:cz[1], cy[0]:cy[1], cx[0]:cx[1]]
rmins = n.array([op['rmin'] for op in all_ops])
rmaxs = n.array([op['rmax'] for op in all_ops])

In [8]:
mov_raw = lbmio.load_and_stitch_tifs(job.tifs[1:2], planes = n.arange(cz[0],cz[1]))[:,:,cy[0]:cy[1], cx[0]:cx[1]]
mov_raw = mov_raw[:,ct[0]:ct[1]]
mov_pos = mov_raw - min_pix_vals.reshape(len(min_pix_vals), 1, 1, 1)[cz[0]:cz[1],]

Loading /mnt/a2/AH007/2022-03-28/2/AH007_ROI3p33um_NOSCREEN_-400um_00002_00001.tif
Loaded 1 files, total 5.68 GB


In [9]:
from suite3d import register_gpu as rgpu 

In [10]:
%%time
mov_shifted, ymaxs, xmaxs = rgpu.rigid_2d_reg_gpu(mov_pos, mask_mul, mask_offset, ref_2ds, shift=True,
                                                 max_reg_xy = 50, rmins=rmins, rmaxs=rmaxs, crosstalk_coeff=crosstalk_coeff)

CPU times: user 4.25 s, sys: 5.22 s, total: 9.46 s
Wall time: 9.48 s


In [11]:
%%time
mov_shifted_gpu = mov_shifted.get()
ymaxs_gpu = ymaxs.get(); 
xmaxs_gpu = xmaxs.get()

CPU times: user 772 ms, sys: 687 ms, total: 1.46 s
Wall time: 1.46 s


In [12]:
# del mov_gpu
# del mov_shifted
mempool = cp.get_default_memory_pool()
mempool.free_all_blocks()

In [13]:
%%time
ref_gpu = cp.asarray(ref_img)

CPU times: user 8.66 ms, sys: 13.3 ms, total: 22 ms
Wall time: 21.7 ms


In [14]:
zf = 1.0/3.0
max_shift_z = 8

In [15]:
%%time
ref_z = cuimage.zoom(ref_gpu, (1,zf,zf), order=1)
ref_zf = cufft.fft(ref_z, axis=0)
ref_zf_c = cp.conjugate(ref_zf)
ref_cf_cn = ref_zf_c / (n.abs(ref_zf_c) + n.complex64(1e-5))

CPU times: user 4.73 ms, sys: 1.47 ms, total: 6.2 ms
Wall time: 5.99 ms


In [16]:
%%time
mov_z = cuimage.zoom(mov_shifted,zoom=(1.0, 1.0, zf,zf), order=1)
mov_z_f = cufft.fft(mov_z, axis=1)
# mov_z_fn = mov_z_f / (n.abs(mov_z_f) + n.complex64(1e-5))

CPU times: user 109 ms, sys: 1.74 ms, total: 111 ms
Wall time: 110 ms


In [19]:
dotnorm = mov_z_fn * ref_cf_cn
pcorr = cufft.ifft(dotnorm,axis=0)
pcorr = n.concatenate([pcorr[-max_shift_z:], pcorr[:max_shift_z+1]])

In [18]:
pcorr.shape

(17, 25, 401, 338)

In [17]:
mov_shifted.dtype

dtype('float32')